<a href="https://colab.research.google.com/github/SergeyShchus/GIS-API/blob/main/app_example/Aggregating_Traffic_Accident_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregating Traffic Accident Data

In this notebook, we will aggregate traffic accident points by using [traffic collisions](https://learngis.maps.arcgis.com/home/item.html?id=79d3e458dcaf486f81c5591a67538179) from Pasadena, CA and [block groups](https://learngis.maps.arcgis.com/home/item.html?id=af363700f8f84c11a02f084e78a296aa) extracted from the Esri Living Atlas. 

We will start by showing how to aggregate traffic collisions by block group. Then we will aggregate traffic collisions by hexbin. The two approaches reveal different patterns to the traffic collision data.

In [28]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

conda install -c esri -c fastai -c pytorch -c anaconda arcgis=1.8.4 ujson scikit-image=0.15.0 pillow=7.1.2 libtiff=4.1.0 fastai=1.0.60 pytorch=1.4.0 torchvision=0.5.0 python=3.7.9 --yes --no-pin


PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2021-11-25 07:25:34--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2021-11-25 07:25:34--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh.2’

     0K .......... .......... .......... .......... ..........  0% 37.6M 1s
    50K .......... .......... .......... .......... ..........  

In [29]:
import sys
sys.path
 
 
import sys
_ = (sys.path
 .append("/usr/local/lib/python3.7/site-packages"))


### Start by connecting to ArcGIS Online

In [30]:
import arcgis

from arcgis.gis import GIS

gis = GIS()

## Aggregating Traffic Collisions by Block Group

We will use the [traffic collisions](https://learngis.maps.arcgis.com/home/item.html?id=79d3e458dcaf486f81c5591a67538179) from Pasadena, CA and [block groups](https://learngis.maps.arcgis.com/home/item.html?id=af363700f8f84c11a02f084e78a296aa). Let's start by grabbing the layer collection from ArcGIS Online.

In [31]:
# Item Added From Toolbar
# Title: Traffic Collisions | Type: Feature Service | Owner: Learn_ArcGIS
traffic = gis.content.get("79d3e458dcaf486f81c5591a67538179")
traffic

<Item title:"Traffic Collisions" type:Feature Layer Collection owner:Learn_ArcGIS>

In [32]:
# Item Added From Toolbar
# Title: USA Block Groups | Type: Feature Service | Owner: esri_dm
block_groups = gis.content.get("af363700f8f84c11a02f084e78a296aa")
block_groups

<Item title:"Pasadena Block Groups" type:Feature Layer Collection owner:Learn_ArcGIS>

### Visualize Layers on Map

In [33]:
traffic_map = gis.map('Pasadena, CA')
traffic_map

MapView(layout=Layout(height='400px', width='100%'))

In [34]:
from google.colab import output
output.enable_custom_widget_manager()

In [35]:
traffic_map.add_layer(block_groups)

In [36]:
traffic_map.add_layer(traffic)

In [37]:
traffic_map

MapView(layout=Layout(height='400px', width='100%'))

### Aggregate by Block Group

We will use the [```aggregate_points```](https://developers.arcgis.com/python/api-reference/arcgis.features.analysis.html?highlight=aggregate#arcgis.features.analysis.aggregate_points) analysis service to aggregate featues by block group.

In [38]:
from arcgis import features
accidents_by_block_group = features.summarize_data.aggregate_points(
                        point_layer = traffic, 
                        output_name="accidents_by_block_group",
                        polygon_layer = block_groups,
                        keep_boundaries_with_no_points=False)

You need to be signed in to use spatial analysis.


AttributeError: ignored

In [13]:
accidents_by_block_group

NameError: ignored

### Add Features to a Map

Once the process is complete, we can add the data to a map as follows.

In [ ]:
traffic_analysis_map_1 = gis.map('Pasadena, CA', zoomlevel=12)
traffic_analysis_map_1

MapView(layout=Layout(height='400px', width='100%'))

In [ ]:
traffic_analysis_map_1.add_layer(accidents_by_block_group)

## Aggregating by Hexbins

### Generating the Hexbins

We can also aggregate the traffic collisions by hexagon bins. To do this, we must first generate the hexagon bins using [```generate_tessellation```](https://developers.arcgis.com/python/api-reference/arcgis.features.manage_data.html?highlight=generate#arcgis.features.manage_data.generate_tessellation).

In [ ]:
from arcgis import features
hexbins = features.manage_data.generate_tessellation(extent_layer=traffic, 
                                bin_size=1, 
                                bin_size_unit='SquareKilometers', 
                                bin_type='HEXAGON', 
                                output_name="Pasadena_Hexbins", 
                                gis=gis)

You can generate a preview of the item.

In [ ]:
hexbins

<Item title:"Pasadena_Hexbins" type:Feature Layer Collection owner:gbrunner_LearnGIS>

### Aggregate by Hexbin

In [ ]:
from arcgis import features
accidents_by_hexbin = features.summarize_data.aggregate_points(point_layer = traffic, 
                                                output_name = "accidents_by_1km_hexbins",
                                                polygon_layer = hexbins,
                                                keep_boundaries_with_no_points=False)

In [ ]:
accidents_by_hexbin

<Item title:"accidents_by_1km_hexbins" type:Feature Layer Collection owner:gbrunner_LearnGIS>

### Add Features to a Map

Once the process is complete, we can add the data to a map as follows.

In [ ]:
traffic_analysis_map_2 = gis.map('Pasadena, CA', zoomlevel=12)
traffic_analysis_map_2

MapView(layout=Layout(height='400px', width='100%'))

In [ ]:
traffic_analysis_map_2.add_layer(accidents_by_hexbin)

## Conclusions

Here we used two different approaches to summarize traffic collisions. Notice how the patterns change when we change the aggregating geometry. These are important considerations to take into account when performing spatial analysis.